In [25]:
import sys
import csv
import pandas as pd
import json
from tqdm import tqdm
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
import stopwordsiso as stopwords
import pickle


class Languages:
    esga = 'ES-GA'
    si = 'SI'
    gb = 'GB'
    hu = 'HU'
    ua = 'UA'
    all = [esga, si, gb, hu, ua]

In [5]:
sentences = {}
for language in Languages.all:
    print('loading', language)
    sentences[language] = pd.read_feather(f'artefacts/pandas_frames/{language}_parlamint_sentences.feather')
metadata = pd.read_feather('artefacts/pandas_frames/parlamint_metadata.feather')

loading ES-GA
loading SI
loading GB
loading HU
loading UA


In [3]:
metadata

,index,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,Subcorpus,Speaker_role,Speaker_MP,Speaker_Minister,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,0,ParlaMint-ES-GA_2017-09-26-DSPG040.u1,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
1,1,ParlaMint-ES-GA_2017-09-26-DSPG040.u2,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
2,2,ParlaMint-ES-GA_2017-09-26-DSPG040.u3,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Regular,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Arias Rodríguez, Raquel",F,1966
3,3,ParlaMint-ES-GA_2017-09-26-DSPG040.u4,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
4,4,ParlaMint-ES-GA_2017-09-26-DSPG040.u5,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365545,213,ParlaMint-UA_2019-11-13-m0.u214,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Олексійович Гончаренко, Олексій",M,1980
1365546,214,ParlaMint-UA_2019-11-13-m0.u215,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983
1365547,215,ParlaMint-UA_2019-11-13-m0.u216,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Володимирівна Геращенко, Ірина",F,1971
1365548,216,ParlaMint-UA_2019-11-13-m0.u217,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983


In [82]:
sentences[Languages.si]

,sentence_id,speech_id,language,text,lemmas
0,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Spoštovane kolegice poslanke in kolegi poslanc...,spoštovan kolegica poslanka in kolega poslanec...
1,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,"Začenjam 8. izredno sejo Državnega zbora, ki s...",začenjati izreden seja državen zbor ki biti on...
2,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,"Obveščen sem, da se današnje seje ne more udel...",obveščen biti da se današnji seja ne moči udel...
3,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Vse prisotne lepo pozdravljam.,ves prisoten lepo pozdravljati
4,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Prehajamo na določitev dnevnega reda 8. izredn...,prehajati na določitev dneven red izreden seja...
...,...,...,...,...,...
3876178,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,S tem prekinjam to točko dnevnega reda.,z ta prekinjati ta točka dneven red
3876179,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,"Prekinjam pa tudi 5. sejo zbora, ki jo bomo na...",prekinjati pa tudi seja zbor ki on biti nadalj...
3876180,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,Lep večer še naprej vsem!,lep večer še naprej ves
3876181,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,( SEJA JE BILA PREKINJENA 5.,seja biti biti prekinjen


In [6]:
for language in tqdm(Languages.all):
    sentences[language].speech_id = sentences[language].speech_id.apply(lambda x: x.replace('.ana', ''))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.06it/s]


In [7]:
speeches = {}
for language in tqdm(sentences):
    with open(f'artefacts/pandas_frames/speech2lemmas_{language}.pkl', 'rb') as f:
        s2l = pickle.load(f)
        for s in list(s2l.keys()):
            s2l[s.replace('.ana', '')] = s2l[s]
            s2l.pop(s)
        speeches[language] = s2l

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.88s/it]


## Prepare data for TF-IDF

### Time period
Based on Bojan's sentiment scores

In [7]:
sentences_gb = sentences[Languages.gb][sentences[Languages.gb].speech_id.str.contains('commons')]
metadata_gb = metadata[metadata.ID.isin(sentences_gb.speech_id.unique())]

In [9]:
may_1_to_32_2020_metadata = metadata_gb[(metadata_gb.Date >= '2020-05-01') & (metadata_gb.Date <= '2020-05-31')]
may_1_to_32_2020_sentences = sentences_gb[sentences_gb.speech_id.isin(may_1_to_32_2020_metadata.ID)]

In [10]:
con_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'CON'].ID.unique())]
lab_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'LAB'].ID.unique())]
'con:', len(con_sentences), 'lab:', len(lab_sentences)

('con:', 10021, 'lab:', 4672)

In [11]:
# get lemma groups by speach_id
lemmas = [
    ('con', ' '.join(con_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
    ('lab', ' '.join(lab_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
]

### By list of speeches

In [67]:
lemmas = []
for speaker_name, speech_ids in tqdm(metadata[metadata.language == Languages.si].groupby('Speaker_name').ID):
    lemmas.append((
        speaker_name,
        ' '.join([speeches[Languages.si][sid] for sid in speech_ids.values if sid in speeches[Languages.si]])
    ))
lemmas = list(sorted(lemmas, key=lambda x: len(x[1])))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 973/973 [00:00<00:00, 2372.28it/s]


#### Limit speakers

In [73]:
lemmas = []
speakers = ['Janša, Ivan Janez', 'Janković, Zoran']
for speaker_name, speech_ids in tqdm(
        metadata[(metadata.language == Languages.si) & (metadata.isin(speakers).Speaker_name)].groupby(
            'Speaker_name').ID):
    lemmas.append((
        speaker_name,
        ' '.join([speeches[Languages.si][sid] for sid in speech_ids.values if sid in speeches[Languages.si]])
    ))
lemmas = list(sorted(lemmas, key=lambda x: len(x[1])))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 483.08it/s]


### Topics per party

In [42]:
lemmas = {}
artefacts_by_topic = Path('artefacts/by_topic')
lemmas_metadata = {}
for topic in ['healthcare', 'eu', 'war']:
    print('processing', topic)
    topic_lemmas = {}
    topic_df = pd.read_feather(artefacts_by_topic / f'{topic}.feather')
    for language in tqdm(Languages.all):
        language_lemmas_metadata = lemmas_metadata.setdefault(language, {})
        topic_lemmas_metadata = language_lemmas_metadata.setdefault(topic, {})
        language_lemmas = []
        if language not in topic_df.columns:
            continue
        topic_speeches = set(topic_df[language].values)
        speech2lemmas = sentences[language].groupby('speech_id').lemmas.apply(lambda x: ' '.join(x))

        for party, party_metadata in metadata[metadata.language == language].groupby('Speaker_party'):
            party = party if party.strip() else '/'
            topic_lemmas_metadata[party] = party_metadata
            ids = [i for i in party_metadata.ID if i in speech2lemmas and i in topic_speeches]
            language_lemmas.append(
                (party, ' '.join(speech2lemmas[ids]))
            )
        language_lemmas = list(sorted(language_lemmas, key=lambda x: len(x[1]), reverse=True))
        topic_lemmas[language] = language_lemmas
    lemmas[topic] = topic_lemmas
print(lemmas.keys())
print(lemmas[list(lemmas.keys())[0]].keys())
print([l for l, _ in lemmas[list(lemmas.keys())[0]]['SI']])
print([l for l, _ in lemmas[list(lemmas.keys())[0]]['GB']])
topic_artefacts = Path('artefacts/by_topic/tf_idf/by_party')

processing healthcare


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.80s/it]


processing eu


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.77s/it]


processing war


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.85s/it]

dict_keys(['healthcare', 'eu', 'war'])
dict_keys(['SI', 'GB', 'HU', 'UA'])
['SDS', 'SD', 'NSi', 'LDS', 'SMC', 'Levica', 'DeSUS', 'LMŠ', 'SNS', 'SLS', 'SAB', 'ZL', 'NP', 'PS', 'Zares', 'ZLSD', 'DL', 'NeP', 'SMS', '/', 'SLS+SKD', 'Lipa', 'Konkretno', 'ZaAB', 'DLGV']
['CON', 'LAB', 'SNP', 'LD', 'DUP', 'PC', 'GP', 'I;CON', 'I', 'SDLP', 'SNP;PAUB', 'LAB;I', 'SNP;I', 'UUP', 'A', 'CON;I', 'UKIP', 'PC;I', 'I;LD', 'IGC', 'IGC;LJ95', 'LJ95;IGC', 'CON;LAB', '/', '64RT', 'BI', 'CB', 'CB;LAB', 'CON;0UBS;NA', 'CON;CB', 'CON;I;LD', 'CON;LD', 'CON;NA', 'CON;NA;CB', 'I;CB', 'IL', 'IL;NA', 'L8TA', 'LAB;NA', 'LD;CB', 'LD;NA', 'LD;NA;CB', 'LI', 'NA', 'NA;CB', 'NA;CON', 'NA;LAB', 'QMZZ', 'UKIP;I', 'UKIP;NA', 'ZKPW']


## Topics per speaker

In [15]:
lemmas = {}
lemmas_metadata = {}
artefacts_by_topic = Path('artefacts/by_topic')
for topic in ['healthcare', 'eu', 'war']:
    print('processing', topic)
    topic_lemmas = {}
    topic_df = pd.read_feather(artefacts_by_topic / f'{topic}.feather')
    for language in tqdm(Languages.all):
        language_lemmas_metadata = lemmas_metadata.setdefault(language, {})
        topic_lemmas_metadata = language_lemmas_metadata.setdefault(topic, {})
        language_lemmas = []
        if language not in topic_df.columns:
            continue
        topic_speeches = set(topic_df[language].values)
        speech2lemmas = sentences[language].groupby('speech_id').lemmas.apply(lambda x: ' '.join(x))

        for party, party_metadata in metadata[metadata.language == language].groupby('Speaker_name'):
            party = party if party.strip() else '/'
            topic_lemmas_metadata[party] = party_metadata
            ids = [i for i in party_metadata.ID if i in speech2lemmas and i in topic_speeches]
            language_lemmas.append(
                (party, ' '.join(speech2lemmas[ids]))
            )
        language_lemmas = list(sorted(language_lemmas, key=lambda x: len(x[1]), reverse=True))
        topic_lemmas[language] = language_lemmas
    lemmas[topic] = topic_lemmas
print(lemmas.keys())
print(lemmas[list(lemmas.keys())[0]].keys())
print([l for l, _ in lemmas[list(lemmas.keys())[0]]['SI']])
print([l for l, _ in lemmas[list(lemmas.keys())[0]]['GB']])
topic_artefacts = Path('artefacts/by_topic/tf_idf/by_speaker')

processing healthcare


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.94s/it]


processing eu


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.95s/it]


processing war


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.97s/it]

dict_keys(['healthcare', 'eu', 'war'])
dict_keys(['SI', 'GB', 'HU', 'UA'])
['Kordiš, Miha', 'Horvat, Jožef', 'Godec, Jelka', 'Möderndorfer, Jani', 'Germič, Ljubo', 'Grims, Branko', 'Janša, Ivan Janez', 'Muršič, Bojana', 'Pojbič, Marijan', 'Tanko, Jože', 'Cukjati, France', 'Mesec, Luka', 'Breznik, Franc', 'Gorenak, Vinko', 'Dimic, Iva', 'Trček, Franc', 'Bah Žibert, Anja', 'Bručan, Andrej', 'Sok, Alojz', 'Brinovšek, Nada', 'Sluga, Janja', 'Zalokar Oražem, Cvetka', 'Irgl, Eva', 'Tomić, Violeta', 'Bevk, Samo', 'Jelinčič, Zmago', 'Pavšič, Robert', 'Jeraj, Alenka', 'Lenart, Jože', 'Tonin, Matej', 'Lisec, Tomaž', 'Vrtovec, Jernej', 'Veber, Janko', 'Barovič, Bogdan', 'Mahnič, Žan', 'Potrata, Majda', 'Siter, Primož', 'Černač, Zvonko', 'Kociper, Maša', 'Žgajner Tavš, Barbara', 'Cigler, Željko', 'Tašner Vatovec, Matej', 'Bizjak Mlakar, Julijana', 'Drobnič, Janez', 'Židan, Dejan', 'Grill, Ivan', 'Cvikl, Milan Martin', 'Sukič, Nataša', 'Žnidarič, Mojca', 'Vizjak, Andrej', 'Peče, Sašo', 'Bratušek, A

## Calculate TF-IDF

In [ ]:
if type(lemmas) is list:
    lemmas['general'] = lemmas

stopwords_for_language = {
    Languages.si: list(stopwords.stopwords('sl')) + ['ter', 'še', 'imeti', 'pač'],
    Languages.hu: stopwords.stopwords('hu'),
    Languages.esga: stopwords.stopwords('es'),
    Languages.gb: stopwords.stopwords('en'),
    Languages.ua: [w.strip() for w in open('data/stopwords_ua.txt', 'r', encoding='utf-8').read().split('\n')],
}


def get_tfidf_dataframe(vectorizer, documents):
    X = vectorizer.fit_transform(documents)

    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

    tfidf_df = tfidf_df.T
    top_words = {}
    for i in range(len(documents)):
        sorted_df = tfidf_df[i].sort_values(ascending=False)
        top_words[language_lemmas[i][0]] = sorted_df.index.tolist()
        top_words[f'{language_lemmas[i][0]}_scores'] = sorted_df.tolist()
    return pd.DataFrame(top_words)


tf_idf = {}
for topic, topic_lemmas in lemmas.items():
    print('processing', topic)
    tf_idf[topic] = {}
    topic_tf_idf = tf_idf[topic]
    for language, language_lemmas in tqdm(topic_lemmas.items()):
        sw = stopwords_for_language[language]
        assert len(sw) > 0
        vectorizer = TfidfVectorizer(stop_words=list(sw), min_df=3, max_df=0.3)
        documents = [l for _, l in language_lemmas]
        topic_tf_idf[language] = get_tfidf_dataframe(vectorizer, documents)
        if topic_artefacts:
            topic_tf_idf[language].to_feather(topic_artefacts / f'tf_idf_{topic}_{language}.feather')

# Calculate sentence scores

In [17]:
def get_sentence_score(sentence_lemmas, tf_idf_scores):
    sentence_lemmas = sentence_lemmas.split()
    scores = []
    for w in sentence_lemmas:
        if w not in tf_idf_scores:
            continue
        scores.append(tf_idf_scores[w])
    return (sum(scores) / len(scores)) if scores else 0


def apply_function(row):
    sentence_id = row['sentence_id']
    text = row['text']
    slemmas = row['lemmas']
    sentence_score = get_sentence_score(slemmas, group_tf_idf_scores)
    return sentence_id, sentence_score, text


tf_idf_sentences = {}
for language, language_sentences in list(sentences.items())[:1]:
    if language == Languages.esga:
        continue
    for topic in tf_idf[:1]:
        print('processing', language, topic)
        topic_tf_idf = tf_idf[topic][language]
        topic_tf_idf_sentences = tf_idf_sentences.setdefault(topic, {})

        group_sentences = {}
        min_len = 1e9
        for group, group_metadata in tqdm(list(lemmas_metadata[language][topic].items())[:1]):
            topic_sentences = language_sentences[language_sentences.speech_id.isin(group_metadata.ID)]
            group_tf_idf_scores = topic_tf_idf.set_index(group)[f'{group}_scores']
            topic_sentences = topic_sentences.apply(apply_function, axis=1)
            topic_sentences = list(sorted(topic_sentences, key=lambda x: x[1], reverse=True))
            group_sentences[f'{group}_ids'] = [s for s, _, _ in topic_sentences]
            group_sentences[group] = [s for _, _, s in topic_sentences]
            group_sentences[f'{group}_scores'] = [s for _, s, _ in topic_sentences]
            min_len = min(min_len, len(topic_sentences))
        group_sentences = {k: v[:min_len] for k, v in group_sentences.items()}
        topic_tf_idf_sentences[language] = pd.DataFrame.from_dict(group_sentences)
        if topic_artefacts:
            topic_tf_idf_sentences[language].to_feather(
                topic_artefacts / f'tf_idf_sentences_{topic}_{language}.feather')
    break

In [44]:
def get_unique_words(df):
    topic_columns = [col for col in df.columns if not col.endswith('_score')]

    unique_words = {}
    all_words = []
    for col in topic_columns:
        all_words.extend(df[col].dropna().tolist())

    for topic in tqdm(topic_columns):
        topic_words = df[topic].dropna().tolist()
        unique_words[topic] = [w for w in topic_words if all_words.count(w) == 1]
    return unique_words


topics = ['tf_idf_eu_GB.feather', 'tf_idf_eu_HU.feather', 'tf_idf_eu_SI.feather', 'tf_idf_eu_UA.feather',
          'tf_idf_healthcare_GB.feather', 'tf_idf_healthcare_HU.feather', 'tf_idf_healthcare_SI.feather',
          'tf_idf_healthcare_UA.feather', 'tf_idf_war_GB.feather', 'tf_idf_war_HU.feather', 'tf_idf_war_SI.feather',
          'tf_idf_war_UA.feather', ]

for topic in topics:
    f = pd.read_feather(topic_artefacts / topic)
    f[0:500].to_csv(topic_artefacts / f'{topic}.csv', index=False)
    unique_words = get_unique_words(f.head(50))
    with open(topic_artefacts / f'{topic}_unique.csv', 'w', encoding='utf-8') as f:
        writer = csv.writer(f)
        columns = unique_words.keys()
        writer.writerow(columns)
        writer.writerows(zip(*[unique_words[column] for column in columns]))



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 104/104 [00:00<00:00, 332.95it/s]
